![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Legal Assertion Status Model

In [0]:
from johnsnowlabs import * 

## Legal Assertion Status Model

The model has been implemented within Spark NLP as an annotator called **AssertionDLModel**. It is an Assertion Status Model aimed to detect **temporality (PRESENT, PAST, FUTURE)** or **certainty (POSSIBLE)** in your legal texts.

## Prediction Pipeline

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings_ner = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base", "en") \
    .setInputCols("sentence", "token") \
    .setOutputCol("embeddings_ner")\

ner_model = legal.NerModel.pretrained('legner_contract_doc_parties', 'en', 'legal/models')\
    .setInputCols(["sentence", "token", "embeddings_ner"])\
    .setOutputCol("ner")

ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setWhiteList(["DOC", "EFFDATE", "PARTY"])

embeddings_ass = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings_ass")

assertion = legal.AssertionDLModel.pretrained("legassertion_time", "en", "legal/models")\
    .setInputCols(["sentence", "ner_chunk", "embeddings_ass"]) \
    .setOutputCol("assertion")


nlpPipeline = nlp.Pipeline(stages=[
            document_assembler, 
            sentence_detector,
            tokenizer,
            embeddings_ner,
            ner_model,
            ner_converter,
            embeddings_ass,
            assertion
            ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

light_model = nlp.LightPipeline(model)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][OK!]
roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]
legner_contract_doc_parties download started this may take some time.
[ | ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]
legassertion_time download started this may take some time.
[ | ][ / ][ — ][ \ ][OK!]

### Getting Result

In [0]:
sample_text = "This is an Intellectual Property Agreement between Amazon Inc. and Atlantic Inc."


In [0]:
data = spark.createDataFrame([[sample_text]]).toDF("text")

data.show(truncate = 80)

+--------------------------------------------------------------------------------+
 text|
+--------------------------------------------------------------------------------+
This is an Intellectual Property Agreement between Amazon Inc. and Atlantic Inc.|
+--------------------------------------------------------------------------------+

In [0]:
result = model.transform(data)

In [0]:
from pyspark.sql import functions as F

result.select(F.explode(F.arrays_zip(result.ner_chunk.result,  
                                     result.ner_chunk.begin, 
                                     result.ner_chunk.end, 
                                     result.ner_chunk.metadata, 
                                     result.assertion.result)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['4']").alias("assertion")).show(truncate=False)

+-------------------------------+-----+---+---------+---------+
chunk |begin|end|ner_label|assertion|
+-------------------------------+-----+---+---------+---------+
Intellectual Property Agreement|11 |41 |DOC |PRESENT |
Amazon Inc |51 |60 |PARTY |PRESENT |
Atlantic Inc |67 |78 |PARTY |PRESENT |
+-------------------------------+-----+---+---------+---------+

In [0]:
from sparknlp_display import AssertionVisualizer

vis = AssertionVisualizer()
viz = vis.display(result.collect()[0], 'ner_chunk', 'assertion',return_html=True)
displayHTML(viz)


This is an Intellectual Property Agreement DOC PRESENT between Amazon Inc PARTY PRESENT . and Atlantic Inc PARTY PRESENT .

### Getting Result with LightPipeline

In [0]:
sample_text = ["""This TRADEMARK LICENSE AGREEMENT (this "Agreement") is made and effective as of 31 Aug, 2020 ("Effective Date"), by and between Palmer Square Capital Management LLC, a Delaware limited liability company (the "Licensor"), and Palmer Square Capital BDC Inc., a corporation organized under the laws of the State of Maryland (the "Licensee")""",
               """The Intellectual Property Agreement would potentially be in short signed by the two Parties""",
               """The Sponsorship Agreement ("Agreement") will be signed on October 10, 2015""",
               """This is an Intellectual Property Agreement between Amazon.com Inc and Atlantic Inc.""",
               """This Sponsorship Agreement ("Agreement") was entered into as of December 18, 1998, by and between Ford Motor Media, a division of J. Walter Thompson with offices at 300 Renaissance Center, Detroit, Michigan 48243 and iVillage Inc., with offices at 170 Fifth Avenue, New York, New York 10010."""]

In [0]:
import pandas as pd

chunks=[]
entities=[]
status=[]
begin = []
end = []

for i in range(len(sample_text)):

    light_result = light_model.fullAnnotate(sample_text[i])[0]

    for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
        begin.append(n.begin)
        end.append(n.end)
        chunks.append(n.result)
        entities.append(n.metadata['entity']) 
        status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'begin':begin, 'end':end, 'ner_label':entities, 'assertion':status})

In [0]:
df

Out[10]:

,chunks,begin,end,ner_label,assertion
0,TRADEMARK LICENSE AGREEMENT,5,31,DOC,PRESENT
1,"31 Aug, 2020",80,91,EFFDATE,PRESENT
2,Palmer Square Capital Management LLC,128,163,PARTY,PRESENT
3,Palmer Square Capital BDC Inc,225,253,PARTY,PRESENT
4,Intellectual Property Agreement,4,34,DOC,POSSIBLE
5,Sponsorship Agreement,4,24,DOC,FUTURE
6,"October 10, 2015",58,73,EFFDATE,FUTURE
7,Intellectual Property Agreement,11,41,DOC,PRESENT
8,Amazon.com Inc,51,64,PARTY,PRESENT
9,Atlantic Inc,70,81,PARTY,PRESENT


### Visualization of Assertion Status

In [0]:
# from sparknlp_display import AssertionVisualizer

vis = AssertionVisualizer()

for i in range(len(sample_text)):
    
    light_result = light_model.fullAnnotate(sample_text[i])[0]    

    viz = vis.display(light_result, 'ner_chunk', 'assertion',return_html=True)
    displayHTML(viz)
    print("\n\n")

This TRADEMARK LICENSE AGREEMENT DOC PRESENT (this "Agreement") is made and effective as of 31 Aug, 2020 EFFDATE PRESENT ("Effective Date"), by and between Palmer Square Capital Management LLC PARTY PRESENT , a Delaware limited liability company (the "Licensor"), and Palmer Square Capital BDC Inc PARTY PRESENT ., a corporation organized under the laws of the State of Maryland (the "Licensee")

The Intellectual Property Agreement DOC POSSIBLE would potentially be in short signed by the two Parties

The Sponsorship Agreement DOC FUTURE ("Agreement") will be signed on October 10, 2015 EFFDATE FUTURE

This is an Intellectual Property Agreement DOC PRESENT between Amazon.com Inc PARTY PRESENT and Atlantic Inc PARTY PRESENT .

This Sponsorship Agreement DOC PAST ("Agreement") was entered into as of December 18, 1998 EFFDATE PAST , by and between Ford Motor Media PARTY PAST , a division of J. Walter Thompson with offices at 300 Renaissance Center, Detroit, Michigan 48243 and iVillage PARTY PAST Inc., with offices at 170 Fifth Avenue, New York, New York 10010.